## Setup


In [ ]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    bbox = font.getbbox(display_str)
    text_width, text_height = bbox[2], bbox[3]
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)

### More images
Perform inference on some additional images with time tracking.


In [ ]:
image_urls = [
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  "https://media.istockphoto.com/id/1266674077/photo/fruts-and-vegetables-at-market.jpg?s=612x612&w=0&k=20&c=HeT7-4YXcAK-t95C2JVPVQVx_6Dkvx0H_LFZsi7xXGQ=",
  "https://media.gettyimages.com/id/155287967/photo/cars-in-rush-hour-with-traffic-at-dawn.jpg?s=612x612&w=gi&k=20&c=A_EXfTmXoKFNLDEDBM7dxT2Hn1456voreYXqaoOgZu0=",
  "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAHsAuAMBIgACEQEDEQH/xAAbAAACAgMBAAAAAAAAAAAAAAAFBgQHAAIDAf/EAEEQAAIBAwIEAwYDBQYEBwAAAAECAwAEEQUhBhIxQRNRYRQicYGRoTKxwQcVQlLRI4KSouHwFiRi8TNEU2NyssL/xAAZAQADAQEBAAAAAAAAAAAAAAACAwQFAQD/xAAnEQACAgICAgIBBAMAAAAAAAABAgADESESMQRBEyJhFFFxsSMyM//aAAwDAQACEQMRAD8Acl5SCMg0s38ns8jIOg6USt70CxWWQBeo+O+KA6hcx3BznB8/9KylyTNUnAkG41FgagS3BkrhfHw5CrHf0qIshzTgmYsuBJLQmTtvUe5sJOUkDFEtPjaRhvt5mmSHTkkVVlQ4Pmw/IZoipWeXDSvbJLtb+3CuRmRfzq9pIJbyJxbtNE6788WOYeu9JVjoVu+rRmMkNGQcEjY9tjg1ZOh2slusxm6lgBv2oUBtfE47CtTF173iHS3iN4t3qdqj82YAI5VHkwIxIvzBo9pfE+naiAkM4M2MmGQeHJ/hNT9Ru4rGDxZQzsxCxxqMtI56KPj/AN6F6taSXmgvb6sscrXDDxIwPdUE55R54x171YeSKSTIvq7YAxCcWp2c8AlSeNRy8zxOwDj0I/Sli7nm1O4e6k544T7lumcYXzPqfyxSveaMltP4VrbypCh5UaOT8K8x2wTvjzpZ4i4k1DQtUgtbGUhBbiSQPnqxOBsQNgB9anNxs+sciBGydy2+GYyNUkLLnEJI26ZYVL1Lll4o0mIj8Mcz/YD+tVDpH7Ub+ykLvbK+RgkEHI+YB+9MmhccRa1xPHcxxN7Y1oYFTGFQAk8565xnpmqKTxAWKuyx5R5hnXRNWXT5Dy2V3l7Zm6I5O8fw7j44opqoP7unGDuvnilex4t065nuLPWprd5raYBCi5IIA6gZwQc9KNX+qWF3p8iQXUTs2Byc2D1HY715hhSMxVFbK34zB0UTgHBkUHrjegOrSzyzsFleXkUd194dRspzg7j50bnjFtaySchdRnKjY4qvOILgG5eScCJIuYQgNnJxkbisS7LkKJfcTGG31NpgLVHUyKSJmU++EAJwc9Ox3pmsL3xbSBmkA5lzgLgEduvequh1JWgtbuPnMzJ77Rk8xOe+eo2HQVYfD0Ki1DXCs3iYcP0U+WBQojq2AYKZMnXd4beKYDmLSkciM3TIxy/UVtZL7NEF58tuWY7cxPU0v3uoSWWsO2p20kdpGcW8qrlBkYyfLbb60QW6j1BUhs5UkR95HQ55V/1/rWqrIK/zC4ty2IUNws0HieeceorKG38/IhCbBRtWVKbSO43iIrx/8zpqxrsSNh5b0LNpMyNCWRFY/i6kVkFz4FpECTzYFdLp+aZCpxE2ObkGSo77VytwIboW6gLWkXxmK55QBgnyGBv9qgWylpQHBCqrO5H8qjJ/p86etRtbJrW5VrZEgVGKyDqT2we5zShZMsfitPHzxyRlMfzAkHO+3ajNmCIrhyBxCUUUsDRsysryAYXOBuM8o32AByTTNpCSTxpIc+EBgbbZ9P8AfelSS7hHiY8UiQyspdc8nOuMDB6bdhTBw5xJYohstQjnCu/MrxqW674+tPa1XGMxdaPXqO2nW3IqukRZm65PSiOr3p0zRLi6BIZUwmBk8xIA277mkixgnu25rvUb2YMxYReOyIBnyXGfnmlniHizUbjWjoqyi306wwHijAwQi9ST1z+tHRapU8fUC+pgRn3LF4Z1y11nWZGuVk9tVSIY2U8scfp/1HufTHSjPEcojitwOnicx+W361UnAOtHTdQudXu7S+u4nzHGLWMOIycHDZ8lC/U0/Ta5b8RadJc28dxAkCsCs6chLAZ8+m9F8gK8bDOPSyNmsepTnEWtalZa/KbC/nhAVMqrZUnGTlTt3pd1TULrU7x7u8k8SZ8AtgDOBgbCp/EVvdzardXEdvJJCWAV4xzDAAHb4UCZyrcsgKt5EYNeAAgkzormi+g6g1hPJcIwD8vLv5dTQUOM1KjQtA3KRnlJouoPcm6RI19rFop9557tC/rzOCfzNWvFYTC6Qgggt05mx18icVS8SS20gyWjcHIKtgj1BFWR+ziSe5NzLLLeXAUqioJiSTgt1JwOg3oH1symm0LrGZY1zf292WiJljWCX3iy/jA64HX7fCg2t6LpOpxq/KJYhnKglSTjHUdDkb/Cp50+ZleS9flTqILYlVP/AMm6t9h6UjambuDVpha3ckCSHnIGSoJ+1Qh15ZAjWqJ+x1GThLRLZAUkAYJtajmLAYznr3J7+RFOtskaW6hE8MbkoRjBzvSDobMl1Dy3LtIyMRK7c2+D28tqL3+m69NHk6mHQnmIw0XyHKR9811UbsicT4/TRjvJraKL/mZIkXvzsAKG2Hs0NxOtmkaqSrBo1xnI/r+dAIBFpd6F1CGKKWROaM5D/wCI4B+9GLR2mMLNLH4hBDCNsj0/KgbJlHAAaOZ1vo+ZDnvtWVrfziOME4IO9ZSmJJ6g6lUaTfmGFIZZgTHzK4zkDPTf0Ndke8eQKqMpJ/E2QB65oVBpd1DZ3FxfWNzDI2PCWSF03z13ArW3v5FYKUbPT3Rmm31/YlY/wbwi4aFb43BvrlI5HKE5I6dflWkVvMVAOMAYHXamjT9PLpG06YkkUE46/Cic1rYWAQzNCgYAl5iBkntvtQIhfR1AsKqSR1Er2CY4DA4PfGRRjRdJeKQSNIp+dGLhbGHURae5GZIBOrkjDDOGwfTY/wB6ttOnSY2cZU808BmwpyEAwMfehet1PGcVkI5Q1ptuFxtkY7HNV3xmmnwcRXsZ8KR5AGY5IIB35SR13qx4LaYhhBG7tjovYfaqv4o4b1M3GoajcInhoxcjny2Nh6/nVFBKbxFeRxb3uPHActueHytuCv8AaEt7uBuB086Z9JhWS95SMiNS7eRPQfc0t8GR+Dw1BAAcAZwcdT5U28N2xS1ubhhu78o2xsB/r9qdURZbmKtzXVjMqzjCeyj1P2C7szNKgL80UxRwObA/CCft2oKYtHm/s/3hfwZOPDvI0nTO+w/i7Z+FTONrS8k4gvbm7tLlI5CyoZIm5OUDGQcMDliMHA2yaCFonlw8qEOxBXmzn3wWAGdxuVxg5wfhVBGDIwdTtLw5BPjwZdHuM7gJK1q++/4TsdqgzaHPYt71pfxAdD4YmT/Lg/auzRFYwo5ebl27ZOCuP4OpUAbCsthdK3NbyzxLzbMjlMjbvgdebGckbbE17cKB5LC4uJxy3cEkjjZXJjYgehH61an7J7C4t9LuJBEWl8RjIEIbGcAdPRaFcLe36nqZgvJ1mjji52blHNnJHUZ/lP8AF5VZ/CkaW8c8caKoZlPurjJr3Hnqe58DmQ7mcGIggZ6HtSvqNsrNkE56dc71ZF/YW94hWaMq5/jTZqVdU4eu4eaS3xcx9cKMOPl3+VZ9/i2ocruX0+TVYMNqLOnOI7rKkExxlOnQ53+zU1W9/KkSLKUPMeVQF7YJz19KULT+xutw4PVxjcdBkg01acjOQMryZwCD8a0A7sVK+uxMlakQMG95IMD8Sq1xcwHb3Sd8eeKM6TB7PbgkEuRkmpstlGwTmUEgHB9ckVHlLRALENyaguz8hE1af+S/xButPI7FYye+ATWVl1IjSMEVg3TlHQ+teV1FGIDncXOBOINe1LVJRqOpS3FjaW7zSrIFwcbDfGep+1DOF7B9UufbJ42xzFj8Sc/rRfgTTuThPWbsAh9Qljs4gT0UZL/5S30FNui6XFZqqJGFAGNhineWcELA8RcgtN0sli5BGGblG+R+tL2qa1LbXrOth7ZCoweRwrIx+2OlSeObqe2mtoLO5kgPKzsY2Kk52HT4H60owz6iXIkuSynrzKDn7UVVFgw6GdtuQ5RhD+qwSNY6dqvgMLiAO3gx5zyuSGHrgEH4rRLT1juL6KaBgypAEVgd+XOenXrjr5UpavxNcoIrSSFZoY195lGGGdvpR/g7UkvZJWiiYBE5TI64yNgB9ifnQ2rgfY7nayHwANRoS7isLpJbqQLBylZGxkKpHU/PFB73UdLf2mMG4u47hWTkjtyq4bY7tjPyFFwFbI5SMihz21nLcmJ3HjBecx53xnGceVJDsF4xzUozcjNtCQ2+mLDgllUAfGn7Trb2fToIT+IIC3xO5pKjtivuqdhtjrQyPiDjzTGKPFp+pRLsMjlbHqQR+VN8Vgp+0T5SFgAsfruz8RskBvQigWo8NaVqbEXulWcwHQtEMj50IT9p8sGBrnDV7bHu8DeIv3Aonpv7QOFb5lRdT9nkbol1E0ZyfXGD8jV4sQyA1OvqCLj9nGiFua19rs2wceDMeVTgjPKcjvUE8AtawXENncwyyPHyxvOhjZGCcoOUIz8x8KsmKS2uk57aeGZfONw35Vo1sCwON6LipghiIk8OcN3OipPzxyNJIApdpRJkDyPKCBnJwc/E0y6IuJuQd2AorJH4cLvk+6pNRdEgCzFwCQoJ+JP+zXsAT2cwpchZOYx/iTZhUM5rshEMwklcKzfwdSw8sV5sxyFKg9Aa6JyCn0GyuCxuU8RiMK/Rl+BFDrrh2W0kD2cnNET0OzA+WRsaZeletiVCjD3TXuI5cpw/68RFQ3LKRE6spQcpz8a0uFYw88JUOPMdR60W1UcsXs3hCSZ3ypxuq7b0IuY5LeNm2B7Y71leSCLDjqaviMDUAe5E0+KS9LmSFF8HC8wySSd/tn717TXpumey2EaPvIfelPmx3P8AT5V5VddOF3JbLctqJ+iWpsuGeH7JgS7xyXkpAzgscLn5MR/do5C6Lspx8sUF0STWdQL3vEAjS7YBFjhTlVUXOO56kk0ZEcjuvKCzZ2GMmobn52ZEuoTgmDEbieb2jW5yCWWPEa+mBv8AfNCpWKqcfi86dNK4F1O5uJZ9buIYVd2ZY4VLSbk4yc4H3pA1k6hpV5cWt5YENG5UskmQfIjI79a1VdK0AJmeaXscldzha25muJpXzsQu/pv+tP3BmnwBBGJFQzMMtjcE9KEaHphFjC8yYLpzspGcEnP5YoprEx03SRLbqfEWRHGO3KwP6Vk2OGsJM0a0KV/mWBBodrGo8R5Jcdc7flSDxVpVza3a6vYAPe2GVeJQR48Xdfj3FGdR/aPp9pkRiPI7vJk/Qb1rY6pJrdst+keBMT/CV6HHQ/CnWOmuETSLCTzmml3sOp2cN5bMDFIB7xHQ+R9aqfjO9a44hkkjdlaIcoZGIYeuRuNsfSn2SzuuHtQvZbRMaZdxu0i82PAm8x6E1VszNc3skn4mlkPL36nb9Kf46jZEV5LHQMduHbu/TR4Wku5pGky+ZH5tu34s9qnR3zQzpO9pZzSIcgyQg71zSNYokhXZY0CgDtgVq4HKa0OCkbEg5t+8kXGtac789xo6pJ/PbycpHr/piiGiarPf3S2mjatfQzFSwiuPfXA9TmlW4AJpq/ZfZq2qXtyw/wDDhCL8WOT/APWlOiDZhBjGvTZOImZodVNq0GPxxjDH7/pRWLxY1KRv4ak5LKPePp6V2YY6Cu3skvkK6BiezmcI1VTkbk9Setds7bVsLSQfy/WsaExqWd0VR3Jrs5OT79K0llS2wH3P8W+Md/yrpzBVLncdBnbJpe1m5LJyJzt4oBONsJ+mT9hS7bBWhYw6qzYwUQRqfEqWjz391nwHIjGBuo6j07fetdQ197jhqS90OFbi5UiWFZhs2PQHyyfXGKX9UutI1m3u9KtLlWu4jgxEbsR/KT1wfKoGg+MkN/aM2FeMLjH4cDFZfzsujNL4FLArJHCP7WtXuNSW11u3t57dsl5IV8N4h58u/Njy617S5oa6XpuvW97rrFbe3fm5UjLtI3YYHbufQetZVyWFxkSN6uJxiW0s6FypyCPIbVF1rVP3Zp8lzC4EqEFAw6nNCtOfVrhZ5oriyuI1maJPEVomYL3yMj7Vrrltqt5YtDNpE+Mc3PBIsq5+Gzf5ah+NxsCaHyodZjtFxRpraZBePNvKnMsIGWz3GP60Gh1+z4gurkRxIBAFB97Oc569s7VU+ppdRxx2jyTW6MSPCmUxAn54zTjwfZtpunjlA53fmLBiAdqZdeWGDCqrrXa9xpeyh5zyqBXDVNL9q0+WJHBJXYGu6XBz78QbPkcfpUhJosYZZACfMGpgBDYmVxHwheGYsqKueven7R7UWFhDCRnw1xUyN4w+S4RfUGt5GAAZGRs+Wd/tXuB7gch1BHE+nvremPaQXZtZSwIlC5xg56VX8vA+v2c6S2z2N40ZDKMch2PpgdqtITIWIIIrYCM5KkA/SjrtdOoL1I/cqqa54hsif3hw9PgdWgbm/LNR14o09iY5/Gt27iSPpVuGEturc1R7uxgnQrdWkEwP86g/nVKec/uTt4dfqVgl7Y3OPBu4mJ7c2D9Ksr9mliY9KlvfEP8AbzN7o6YXAB/P60C1HgfQboAtYCJz1MbFR8gNqIaTAdCtorbTpJI44RyqCc5+PnRt5qnTCAPBPox8Za73FstwY2PYbml2z4ijdvDvU8M4/Gm4+flRpGF3GriVZYT05Pwn4+fzqpLVs2pkr1PXphNkaKIkWqBm6GQ9B/X5fWvOUyODIxd+zHt8BWx6ADt0ryZvAiZj+IjbHXHemRcHXLSLdzPLIPZI0wqr1I7n4k7D0HrVf8ea7Lp1q0Vvj96XwPIF38NOhb5dB/oaauJtSh0y0kluW5Yoh4s7Du3RVHrVLzahPqF/c6ne5W4l2RQdkXsg+A/WpXPyPj0P7lS/40z7P9Qd7PEiIrxsHXowPvA+efOmXQeJJ9LZf31bG/tCceODiRB/+vn9aERWrSjIb5URFuIx+IYGAxPTejNQcbEWLWQ6hfiCLT9eT94afKkkWQvKBgr8R2NZQOygP7zUaerF8hDyts7Ht6gf77VlR/G1egZV8i2bIjFoWptDaQW/TlXP1JP6032GqZwM/OkeO18J8jYdqL2LNkYp6tgYiCI/W8q3EfLIqsp6qwyDXJuGNEmJYWMcLt1e1Jhb6oRUfSmPJvRmJ8CmYBG4OSOoq8Q6NeaVAtzpepXBgXaaOdVmCDswJAOPnUPTZNXlblCWNwy/yu0TMPgcj7043koMTqQCCMEHvSkkR068VYTiMnMJPb/pqKxF566lSWsF2ZKuJLn/AM5pmoQovUxKJVPxKEnHyrS01TTsLbreRLIBsrnkb6NvTLBccyA7V5PHb3SGO5hilQ9VkQMD9aYfHBGjOL5J9iCfxKGJVvXNbxyuowDzL5EdK9k4a0ljzQQPaN15rSZ4vspx9qiy6Hewg+xay+fK7gSQfVeU/elfA46Mb+oQ9iSUk5iPeA+Q/St2Yk4269fKhrRa5Ax5rS0uVA2ME5Q/4XGP81cW1eSAAXunX9t/1Pbll/xJzCgKN7EYHU9GGG3YZf8ApXKWMOM8gIz8M0Pt9Y0+6JW3vYncHoJACPl1qZ4xY7HJIpTYjR+JHnt0zlEYfAbV7bPPbsWt5WjOdyrbfTv9K7SBioHMcjvjrXPDBcyMc9s9DXlHtTPNvRhiDiOKKNWvYpGxs0kKlsevKN/pmpQvEltmvUvIbu2A/smj/ibyONtvlS5GxDbqAexrYhVJKIqs5HOQMcxHTNWL5L44nuSN4qE5GpXX7RtUuLrV0spDm1iHiMqneRz3Pw8vX4UChkWVedCpGMDbpVqavw7puuRt7VGBKMYlTZgarrXOFdT0KQ3EPNPb/wDrxrvj1FdpvA+rQL6GJ5LNLTORnzrfXLoJCun20Y8aQ43HTvmhttqhjUhwAc/i6/8AajnBWjSa7qjXVyreEMNJ6L2X4n8qrawKuZGtZZsRv/Z/w8tpZpfzoCzA+BzdQD1b4nf5fGspod3URxQAcxIVF6egrKzS7WHIE01RaxgmIZiPlU3T4zz7iiDRJ/KK2iRQ2wFWcZDmGrDZfKikZ2oVZ9BRWHpTBBM53EZYEUImtfElBbflOR8aPSdKjlVz0oDWCZ3kcYkS2JA5aliuWAHOBW/YV0jAgjuY0hWvDPmo8xOetcOY5O/auZjcSYLjyroJwe9Cyx869RjnrXszmJIvrOxvVxe2kE4/9yNW/Oh//DWmHa2a6sz29muGUD+6cr9qkM7eZrtbsc9aEgHuECR0ZBOh6pCMWWsLKPK8gDE/3kK/kaiXJ1mwOLqxtp1G+bS5yceZDhcfWmRGPMN+9LHEDtPrFlZSsTbXFwVljzjnAHQ+npS7KkAziMrufOMzLfiOxniDypdQLnHPNbsE/wAQBX6GiFrdWV8ubK6gn8/ClDflRSJVjkEcaqiIMKqjAA+FSJNG0vUQjX2nWk7fzvCpYfA4yKH9LrRhfqv3EEBrmB0WNFaM5MnO2CPLHnUrKvEyyxnlYYIIBFK3HUX/AA7bvNo8txbsDsPHd1HwViQPpUvg3U7zUbYG8lEh8+RR+QpFlZXuPSwOMgQZxDwFZahIJtNkFpIT76jGCPgf9/CmHQtLj0fTlt0YFusjdC7HvRZwN6jzKBjGRtnY0o2NjHqMVFzmT9Dgae8aaRTyQDClh1Yj9B+de0F4b1a+biA2DT5tcZ5Ci9evXGfvWVreMoWsYmV5LFrDmf/Z"

  ]

def detect_img(image_url):
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",end_time-start_time)

In [ ]:
for i, image_url in enumerate(image_urls):
  detect_img(image_url)
